In [1]:
import pycaret
import pandas as pd
import os
import sys 
import shutil

In [36]:
sys.path.append(os.path.abspath("/Users/raj2.gaurav/Desktop/Git/Causal_Model/09. Config"))
import config

In [3]:
# import pycaret classification and init setup
from pycaret.classification import *

In [4]:
path=config.train_data_path

In [5]:
data=pd.read_pickle(path)

In [6]:
data.shape

(343124, 217)

In [7]:
data.columns

Index(['entity_cell_id', 'hour_of_day', 'entity_geohash',
       'context_environment', 'avg_last_rrc_measurement_earfcn',
       'num_distinct_imsi', 'mute_call_gap_indicator', 'num_mute_calls',
       'avg_cqi', 'min_cqi',
       ...
       'env_indoor_pct', 'env_outdoor_pct', 'env_mobile_pct',
       'env_unknown_pct', 'service_type_data_only_pct',
       'service_type_voice_and_data_pct', 'service_type_unknown_pct',
       'qci_9_tp', 'data_date', 'Target_Variable'],
      dtype='object', length=217)

In [8]:
Target = config.Target
Train_size = config.Train_size
Ignore_features = config.Ignore_features
Preprocess = config.Preprocess
Imputation_type = config.Imputation_type
numeric_imputation = config.numeric_imputation
categorical_imputation = config.categorical_imputation
remove_outliers = config.remove_outliers
outliers_method = config.outliers_method
outliers_threshold = config.outliers_threshold
fix_imbalance = config.fix_imbalance
fix_imbalance_method = config.fix_imbalance_method
normalize = config.normalize
normalize_method = config.normalize_method

In [9]:
s = setup(data, target = Target, train_size = Train_size, ignore_features = Ignore_features,
          preprocess = Preprocess,imputation_type = Imputation_type, numeric_imputation = numeric_imputation,
         categorical_imputation = categorical_imputation, remove_outliers = remove_outliers, 
          outliers_method = outliers_method, outliers_threshold = outliers_threshold,
         fix_imbalance = fix_imbalance, fix_imbalance_method = fix_imbalance_method,
         normalize = normalize, normalize_method = normalize_method)

,Description,Value
0,Session id,2593
1,Target,Target_Variable
2,Target type,Binary
3,Original data shape,"(343124, 217)"
4,Transformed data shape,"(300214, 211)"
5,Transformed train set shape,"(197276, 211)"
6,Transformed test set shape,"(102938, 211)"
7,Ignore features,8
8,Numeric features,207
9,Categorical features,1


In [10]:
report=pull()

In [15]:
Data_Sense=report.iloc[1:10]

In [12]:
models = config.models

In [14]:
best_model = compare_models(include = models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9915,0.9995,0.9948,0.9950,0.9949,0.9696,0.9696,59.8290
rf,Random Forest Classifier,0.9695,0.9944,0.9851,0.9785,0.9818,0.8893,0.8895,17.8380
gbc,Gradient Boosting Classifier,0.9686,0.9933,0.9808,0.9815,0.9811,0.8876,0.8876,64.3820
dt,Decision Tree Classifier,0.9582,0.9268,0.9741,0.9757,0.9749,0.8507,0.8508,8.3190
ada,Ada Boost Classifier,0.9277,0.9723,0.9517,0.9610,0.9563,0.7453,0.7457,105.5260
lr,Logistic Regression,0.8987,0.9703,0.8891,0.9880,0.9360,0.6970,0.7197,74.6910
svm,SVM - Linear Kernel,0.8552,0.0000,0.8350,0.9895,0.9055,0.6045,0.6469,82.3880


Processing:   0%|          | 0/33 [00:00<?, ?it/s]

In [16]:
Compare_Model_Report=pull()

In [19]:
if not os.path.exists(config.data_hygeine_path):
    os.makedirs(config.data_hygeine_path)
writer2 = pd.ExcelWriter(config.data_hygeibe_report_file)
Data_Sense.to_excel(writer2, sheet_name = config.data_hygeibe_report_file_sheet1, index = False)
Compare_Model_Report.to_excel(writer2, sheet_name = config.data_hygeibe_report_file_sheet2, index = False)
writer2.close()

In [21]:
best_5=Compare_Model_Report.head()

In [23]:
model1=create_model(best_5.index[0],fold=config.fold_n)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9910,0.9994,0.9947,0.9944,0.9946,0.9676,0.9676
1,0.9916,0.9995,0.9948,0.9952,0.9950,0.9700,0.9700
2,0.9909,0.9995,0.9949,0.9942,0.9945,0.9672,0.9672
3,0.9915,0.9995,0.9951,0.9947,0.9949,0.9696,0.9696
4,0.9916,0.9995,0.9951,0.9948,0.9950,0.9700,0.9700
Mean,0.9913,0.9995,0.9949,0.9946,0.9948,0.9689,0.9689
Std,0.0003,0.0001,0.0002,0.0003,0.0002,0.0012,0.0012


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
Model1_Report=pull()

In [26]:
tuned_model = tune_model(model1,fold=config.fold_tuned_n)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9932,0.9997,0.9970,0.9948,0.9959,0.9755,0.9755
1,0.9942,0.9998,0.9970,0.9960,0.9965,0.9790,0.9791
2,0.9939,0.9998,0.9969,0.9958,0.9964,0.9781,0.9781
3,0.9934,0.9997,0.9963,0.9958,0.9960,0.9763,0.9763
4,0.9943,0.9998,0.9969,0.9963,0.9966,0.9797,0.9797
Mean,0.9938,0.9998,0.9968,0.9957,0.9963,0.9777,0.9777
Std,0.0004,0.0000,0.0003,0.0005,0.0003,0.0016,0.0016


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM

In [27]:
Model1_Tuned_Report=pull()

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9


In [39]:
Hyper_para=str(tuned_model)
Model1_Hyperparamters=pd.DataFrame(Hyper_para.split('(')[1].split())

In [29]:
model_name = best_5['Model'][0]

In [40]:
if not os.path.exists(eval(config.model_hygeine_path)):
    os.makedirs(eval(config.model_hygeine_path))
writer2 = pd.ExcelWriter(eval(config.model_hygeine_report_file))
Model1_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet1, index = False)
Model1_Tuned_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet2, index = False)
Model1_Hyperparamters.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet3, index = False)
writer2.close()

In [ ]:
### Accuracy results

In [44]:
lst_plot=config.plot_lst
for i in lst_plot:
    plot_model(tuned_model, plot = i,save=True) 

In [46]:
if not os.path.exists(eval(config.plot_path)):
    os.makedirs(eval(config.plot_path))


source = os.path.join(config.plot_soruce_path)
sort = os.path.join(eval(config.plot_path))

for f in os.listdir(source):
    if f.endswith(config.plot_format):
        shutil.move(os.path.join(source, f), sort)

In [47]:
finalize_model(tuned_model)

Pipeline(memory=FastMemory(location=/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['hour_of_day',
                                             'avg_last_rrc_measurement_earfcn',
                                             'num_distinct_imsi',
                                             'mute_call_gap_indicator',
                                             'num_mute_calls', 'avg_cqi',
                                             'min_cqi', 'max_cqi', 'p05_cqi',
                                             'p10_cqi', 'p50_cqi', 'p90_cqi',
                                             'p95_cq...
                                boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                             

In [48]:
if not os.path.exists(eval(config.save_model_path)):
    os.makedirs(eval(config.save_model_path))

In [50]:
save_model(tuned_model, f'{eval(config.save_model_path)}/{best_5.index[0]}', verbose=True)

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['hour_of_day',
                                              'avg_last_rrc_measurement_earfcn',
                                              'num_distinct_imsi',
                                              'mute_call_gap_indicator',
                                              'num_mute_calls', 'avg_cqi',
                                              'min_cqi', 'max_cqi', 'p05_cqi',
                                              'p10_cqi', 'p50_cqi', 'p90_cqi',
                                              'p95_cq...
                                 boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, feature_fraction=0.7,
                                 importance_type='split', learning_rate=0.2,
               

In [51]:
tuned_model

LGBMClassifier(bagging_fraction=0.9, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.2, max_depth=-1,
               min_child_samples=61, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=150, n_jobs=-1, num_leaves=40, objective=None,
               random_state=2593, reg_alpha=0.1, reg_lambda=3, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [52]:
 f'{eval(config.save_model_path)}/{best_5.index[0]}'

'/Users/raj2.gaurav/Desktop/Git/Causal_Model/07. Model/Light Gradient Boosting Machine/Model_Results/Model_Objects/lightgbm'

In [57]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open( f'{eval(config.save_model_path)}/{best_5.index[0]}.pickle', 'wb') as files:
    pickle.dump(tuned_model, files)

In [56]:
from joblib import Parallel, delayed

import joblib

# Save the model as a pickle in a file

joblib.dump(tuned_model,  f'{eval(config.save_model_path)}/{best_5.index[0]}.pkl')

['/Users/raj2.gaurav/Desktop/Git/Causal_Model/07. Model/Light Gradient Boosting Machine/Model_Results/Model_Objects/lightgbm.pkl']